In [1]:
import numpy as np
import pandas as pd

from datetime import date
import warnings
import pandas_market_calendars as mcal

from statsmodels.tsa.stattools import coint

In [2]:
warnings.filterwarnings('ignore')
data_file=pd.read_csv('./data/stock_database.csv')
#data_file.head()

In [3]:
data_file['Date']=data_file['Date'].astype('datetime64[ns]')
data_file.set_index('Date', inplace=True)
data_file.drop(['ARNC', 'FOX', 'FOXA', 'PX', 'SCG', 'AKAM', 'CHK'], axis=1, inplace=True)

smoothing='no'
data_file.to_csv('./data/selected.csv')


In [4]:
#start='1990-01-01'
#end='2020-11-06'
#nyse = mcal.get_calendar('NYSE')
#time_range=nyse.schedule(start_date=start, end_date=end)
#mcal.date_range(time_range, frequency='1W')

In [5]:
sp_list=pd.read_csv('./data/sp_list.csv')
sp_list=sp_list[['Symbol', 'Sector']].copy()

In [6]:
temp_list=[]
main_list=[]

stock_list=data_file.columns.to_list()
for i in range(0, len(stock_list)-1):
    for j in range(i+1, len(stock_list)):
        pair=data_file.iloc[:,[i,j]]
        pair.dropna(how='any', axis=0, inplace=True)
        
        if smoothing=='yes':
            pair=pair.rolling(window=5).mean()
            pair.dropna(inplace=True)
        
        try:
            result=coint(pair.iloc[:,0], pair.iloc[:,1])
        except ValueError:
            print(f'i:{i},j:{j}')
            
        temp_list=[result[0], result[1], result[2][0], result[2][1], result[2][2], i, j]
        main_list.append(temp_list)

test_results=pd.DataFrame(main_list, columns=('t_stat', 'p_value', 'CI_1', 'CI_5','CI_10', 'i', 'j'))

In [7]:
test_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111156 entries, 0 to 111155
Data columns (total 7 columns):
t_stat     111156 non-null float64
p_value    111156 non-null float64
CI_1       111156 non-null float64
CI_5       111156 non-null float64
CI_10      111156 non-null float64
i          111156 non-null int64
j          111156 non-null int64
dtypes: float64(5), int64(2)
memory usage: 5.9 MB


In [8]:
coint_results=test_results.loc[test_results.p_value<0.005]
coint_results=coint_results.sort_values(by='p_value')


In [9]:
new_cols=['i_sybl', 'j_sybl', 'i_sect', 'j_sect']
coint_results=coint_results.reindex(columns=coint_results.columns.tolist()+new_cols)

stock_list=data_file.columns.tolist()
for num in range(0, len(coint_results.index)):

    i=coint_results.iloc[num, 5]
    j=coint_results.iloc[num, 6]
    
    coint_results['i_sybl'].iloc[num]=stock_list[i]
    coint_results['j_sybl'].iloc[num]=stock_list[j]

coint_results=pd.merge(coint_results, sp_list, how='left', left_on='i_sybl', right_on='Symbol')
coint_results['i_sect']=coint_results['Sector']
coint_results.drop(columns=['Sector'], axis=1, inplace=True)
coint_results=pd.merge(coint_results, sp_list, how='left', left_on='j_sybl', right_on='Symbol')
coint_results['j_sect']=coint_results['Sector']
coint_results.drop(columns=['Symbol_x', 'Symbol_y', 'Sector'], axis=1, inplace=True)
coint_results.head()    

,t_stat,p_value,CI_1,CI_5,CI_10,i,j,i_sybl,j_sybl,i_sect,j_sect
0,-10.181139,8.693839e-17,-3.898623,-3.337348,-3.045295,188,195,GLW,GRMN,Information Technology,Consumer Discretionary
1,-10.144893,1.071887e-16,-3.899871,-3.338043,-3.045778,19,350,AIG,PM,Financials,Consumer Staples
2,-9.915226,4.061805e-16,-3.899873,-3.338044,-3.045779,19,435,AIG,V,Financials,Information Technology
3,-9.182841,2.967665e-14,-3.898584,-3.337326,-3.045280,188,225,GLW,ILMN,Information Technology,Health Care
4,-8.627555,7.775643e-13,-3.899712,-3.337955,-3.045716,156,409,ETFC,TEL,Financials,Information Technology


In [10]:
if smoothing=='yes':
    coint_results.to_csv('./data/results_smoothing.csv', index=False)
else:
    coint_results.to_csv('./data/results.csv', index=False)